In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

# New Section

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/',  force_remount=True)
# Sort files in folder
from natsort import natsorted, ns

#(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
dataset_path = r"/content/drive/My Drive/Colab Notebooks/Math_Solver/assets/pruned_dataset/"
  #r"/content/drive/My Drive/Math_Solver/assets/pruned_dataset/" worked for Enrique
subtract_testing = [x for x in natsorted(os.listdir(dataset_path + "-/testing/"))]
subtract_training = [x for x in natsorted(os.listdir(dataset_path + "-/training/"))] 

# Normalize pixel values to be between 0 and 1 KEEP THIS!!!
train_images, test_images = train_images / 255.0, test_images / 255.0


Mounted at /content/drive/


NameError: ignored

In [ ]:
# change as fit/needed

model = models.Sequential() # creates sequential model
# below are the network layers being added
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1))) # no rgb grey black n white
model.add(layers.MaxPooling2D((2, 2))) 
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))# 10 sybols/ charcters 

model.summary()

# change optimizer, loss, and metrics
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# change epochs as needed for higher accuracy
history = model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels), batch_size=1)


In [ ]:
# evaluate model to see if the model is improving
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
!pip install jupyter-dash
!pip install dash-canvas==0.1.0
!pip install dash==1.17.0

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
from dash_canvas import DashCanvas
import plotly.express as px
from jupyter_dash import JupyterDash


In [ ]:

filename = 'https://drive.google.com/file/d/1hYD2JiDmR6qLFwUbb7pO2d8KEg0ToMU8/view?usp=sharing'
# external_script = 'https://drive.google.com/file/d/1KzrU7av15opjPjZnPomXFQtPdMJb960j/view?usp=sharing'
# external_script = r"/content/drive/My Drive/Colab Notebooks/Math_Solver/assets/app.js" # not entirely sure if I am doing this correctly -AJ
# external_css = 'https://drive.google.com/file/d/1-16FYwgI4_7Ocyg7Fmjs_eJQQJbqtB7J/view?usp=sharing'


app = JupyterDash(__name__)

app.layout = html.Div(html.Center([
    html.H1("My Math Solver",style={"color":"cornflowerblue", "textAlign":"center"}),
    html.Div(
      DashCanvas(id='canvas', 
                 hide_buttons=['zoom', 'line', 'select', 'rectangle'],
                 lineColor='black',
                 lineWidth=5,
                 width=400,
                 filename=filename),
                 style={"margin":"auto","border":"2px solid black", "width": "400px"}
    ),
    html.Div(
        dcc.Input(
            id='text_field', 
            type='text',
            placeholder="Enter equation here!")
        ),
    html.Button('Submit', id='submit_button', n_clicks=0),
    html.Div(id='container-button-basic',
             children='Enter a equation and press submit'),
    html.Img(id='my-image', width=300),
    html.P(id='test-p', className='test-p', children='Some test string to see if css is working')
  ]))

@app.callback(Output('my-image', 'src'),[Input('canvas', 'json_data')])
def update_data(string):
    if string:
        mask = parse_jsonstring(string, io.imread(filename, as_gray=True).shape)
    else:
        raise PreventUpdate
    return array_to_data_url(img_as_ubyte(src))

app.run_server(mode='external')

In [ ]:
# Load Data
# df = px.data.tips()
# filename = 'https://drive.google.com/file/d/1hYD2JiDmR6qLFwUbb7pO2d8KEg0ToMU8/view?usp=sharing'
# contains javascript function to install image
# external_script = 'https://drive.google.com/file/d/1KzrU7av15opjPjZnPomXFQtPdMJb960j/view?usp=sharing'
# external_css = 'https://drive.google.com/file/d/1-16FYwgI4_7Ocyg7Fmjs_eJQQJbqtB7J/view?usp=sharing'
# Build App
# app = JupyterDash(__name__, assets_external_path=external_css, external_stylesheets=external_css)
app = JupyterDash(__name__)
app.layout = html.Div(html.Center([
    html.H1("My Math Solver",style={"color":"cornflowerblue", "textAlign":"center"}),
    html.Div(
      DashCanvas(id='canvas', 
                 hide_buttons=['zoom', 'line', 'select', 'rectangle'],
                 lineColor='black',
                 lineWidth=5,
                 width=400),
                 style={"margin":"auto","border":"2px solid black", "width": "400px"}
    ),
    html.Div(
        dcc.Input(
            id='text_field', 
            type='text',
            placeholder="Enter equation here!")
        ),
    html.Button('Submit', id='submit_button', n_clicks=0),
    html.Div(id='container-button-basic',
             children='Enter a equation and press submit'),
    html.Img(id='my-image', width=300),
    html.P(id='test-p', className='test-p', children='Some test string to see if css is working')
  ]))

@app.callback(Output('my-image', 'src'),[Input('canvas', 'json_data')])
def update_data(string):
    if string:
        mask = parse_jsonstring(string, io.imread(filename, as_gray=True).shape)
    else:
        raise PreventUpdate
    return array_to_data_url(img_as_ubyte(src))

# app.run_server(mode='external')
# Run app and display result inline in the notebook
# app.css.append_css(dict(external_url=external_css))
# app.run_server(mode='external', host='127.0.0.1', port=8050, debug=True, use_reloader=False)

In [ ]:
app._terminate_server_for_port('127.0.0.1', 8050)

In [ ]:
# Do not RUN! production Server only!!!
# ! pip install pyngrok
# ! ngrok authtoken xxxxxxx

# from pyngrok import ngrok

# # Open Http tunnel to port
# public_url = ngrok.connect(port = '8050')
# public_url
# ngrok.kill()
